In [1]:
# This link might be useful, unclear yet: https://medium.com/randomai/ensemble-and-store-models-in-keras-2-x-b881a6d7693f
import numpy as np
import keras
from keras import backend
from keras.models import load_model
import tensorflow as tf
from utils import *

import cleverhans.attacks as Attacks
from cleverhans.attacks import FastGradientMethod
from cleverhans.attacks import BasicIterativeMethod
from cleverhans.utils_keras import KerasModelWrapper

/home/oyku/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
ATTACK_NAME = 'fgsm' # sys.argv[1]
DATASET_NAME = 'mnist' # sys.argv[2]

In [3]:
KNOWN_SEED = 87

# l-inf attacks
if ATTACK_NAME == 'fgsm':
    AttackModel = Attacks.FastGradientMethod
    attack_params = {
    'eps': 0.5,
    'clip_min': 0.0,
    'clip_max': 1.0
    }
elif ATTACK_NAME == 'pgd':
    AttackModel = Attacks.ProjectedGradientDescent
    attack_params = {
    'eps': 0.5,
    'clip_min': 0.0,
    'clip_max': 1.0
    }
elif ATTACK_NAME == 'bim':
    # subclass of pgd (rand_init == 0)
    AttackModel = Attacks.BasicIterativeMethod
    attack_params = {
    'eps': 0.5,
    'clip_min': 0.0,
    'clip_max': 1.0
    }
# l-2 
elif ATTACK_NAME == 'inf-pgd':
    AttackModel = Attacks.ProjectedGradientDescent
    attack_params = {
    'eps': 0.5,
    'clip_min': 0.0,
    'clip_max': 1.0,
    'ord': np.inf
    }
elif ATTACK_NAME == 'cw':
    # subclass of pgd (rand_init == 0)
    AttackModel = Attacks.CarliniWagnerL2
    attack_params = {
    'eps': 0.5,
    'clip_min': 0.0,
    'clip_max': 1.0
    }




In [4]:
dataset = DATASET_NAME
num_classes = 10
(x_train, y_train, x_test, y_test) = get_dataset(dataset)

x_train shape (60000, 28, 28, 1)
y_train shape (60000,)
x_test shape (10000, 28, 28, 1)
y_test shape (10000,)


In [5]:
def permute_pixels(images, seed):
    n_sample, img_r, img_c, n_channels = images.shape
    np.random.seed(seed)
    perm_idx = np.random.permutation(img_r*img_c)
    permuted_images = np.zeros_like(images)
    for idx in range(n_sample):
        for channel in range(n_channels):
            img = images[idx, :, :, channel].flatten()
            img = img[perm_idx]
            permuted_images[idx, :, :, channel] = img.reshape((img_r, img_c))
    return permuted_images

In [6]:
backend.set_learning_phase(False)
sess =  backend.get_session()

# Define input TF placeholder
if DATASET_NAME == 'mnist':
    x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
    y = tf.placeholder(tf.float32, shape=(None, 10))
elif DATASET_NAME == 'cifar10':
    x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3))
    y = tf.placeholder(tf.float32, shape=(None, 10))

In [9]:
# this is the secret seed 87 one
# we're going to give this trained model to the adversary

keras_model = load_model('models/'+dataset+'_trained_keras_model'+'.hdf5', custom_objects={'tf':tf}) 
x_shuffle = permute_pixels(x_test, KNOWN_SEED)
pred = np.argmax(keras_model.predict(x_shuffle), axis = 1)
acc =  np.mean(np.equal(pred, y_test))
print("The normal test accuracy is: {}".format(acc))

The normal test accuracy is: 0.957


In [13]:
# generate adversariale examples (x_adv) using the 87 keras model
# http://everettsprojects.com/2018/01/30/mnist-adversarial-examples.html
# https://cleverhans.readthedocs.io/en/latest/source/attacks.html#generate_np
wrap = KerasModelWrapper(keras_model)
attack_model = AttackModel(wrap, sess=sess)
x_adv = attack_model.generate_np(x_test, **attack_params)

# test x_adv against the single model
x_shuffle = permute_pixels(x_adv, KNOWN_SEED)
pred = np.argmax(keras_model.predict(x_shuffle), axis = 1)  # predicted class labels
acc =  np.mean(np.equal(pred, y_test))
print("The adversarial validation accuracy is: {}".format(acc))

The adversarial validation accuracy is: 0.5132


In [14]:
# test x_adv against the ensemble model

num_models = 1  # debugging with smaller number. change this to 50 later.
num_samples = x_adv.shape[0]
acc_per_model = []
# We're assuming majority voting?
pred_all_model = np.zeros((num_samples, num_classes)) # we're going to store votes from each model here

for SECRET_SEED in range(num_models):
    keras_model = load_model('models/'+dataset+'_trained_keras_model_'+str(SECRET_SEED)+'.hdf5', custom_objects={'tf':tf})
    x_adv_shuffle = permute_pixels(x_adv, SECRET_SEED)
    pred = np.argmax(keras_model.predict(x_adv_shuffle), axis = 1)  # predicted class labels
    pred_all_model[:, pred] += 1  # +1 vote 
    acc =  np.mean(np.equal(pred, y_test))
    print ('Individual model accuracy:', acc, 'SECRET_SEED:', SECRET_SEED)
    acc_per_model.append(acc)  # accuracy per model, not reported in paper

# for each adversarial sample, find out the class with most votes
ensemble_pred = np.argmax(pred_all_model, axis = 1)
acc =  np.mean(np.equal(ensemble_pred, y_test))
print ('Ensemble accuracy:', acc)

OSError: Unable to open file (unable to open file: name = 'models/mnist_trained_keras_model_0.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)